In [25]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
import spotipy.client as client

import json
import time

import requests

In [26]:
songs_df = pd.read_csv('my_data/songs_billboard.csv')
artists_df = pd.read_csv('my_data/artistes_billboard.csv')
billboard = pd.read_csv('my_data/my_billboard.csv')
spotify_df = pd.read_csv('my_data/songs_spotify_features.csv')

In [27]:
songArtistSongID=billboard[['Song','Performer','SongID']].drop_duplicates()
songArtistSongID=songArtistSongID.reset_index()
songArtistSongID.head()

Song  \
0                                   Poor Little Fool   
1                                      One Sweet Day   
2  Candle In The Wind 1997/Something About The Wa...   
3                                Do I Make You Proud   
4                                                  3   

                    Performer  \
0                Ricky Nelson   
1  Mariah Carey & Boyz II Men   
2                  Elton John   
3                Taylor Hicks   
4              Britney Spears   

                                              SongID  
0                       Poor Little FoolRicky Nelson  
1            One Sweet DayMariah Carey & Boyz II Men  
2  Candle In The Wind 1997/Something About The Wa...  
3                    Do I Make You ProudTaylor Hicks  
4                                    3Britney Spears

In [28]:
print(len(songArtistSongID),len(spotify_df))

28475 28475


In [29]:
spotify_df = pd.merge(songArtistSongID,spotify_df,how='left')

In [30]:
spotify_df.tail()

Song  \
28470                 What'd You Come Here For?   
28471                     What's A Guy Gotta Do   
28472  What's A Matter Baby (Is It Hurting You)   
28473        Everybody Dies In Their Nightmares   
28474                  Into The Great Wide Open   

                             Performer  \
28470                   Trina & Tamara   
28471                      Joe Nichols   
28472                        Timi Yuro   
28473                     XXXTENTACION   
28474  Tom Petty And The Heartbreakers   

                                                  SongID  year  \
28470            What'd You Come Here For?Trina & Tamara  1999   
28471                   What's A Guy Gotta DoJoe Nichols  2005   
28472  What's A Matter Baby (Is It Hurting You)Timi Yuro  1962   
28473     Everybody Dies In Their NightmaresXXXTENTACION  2018   
28474  Into The Great Wide OpenTom Petty And The Hear...  1991   

                                           spotify_genre  \
28470  ['deep pop r&b', 'hip pop', 'miami hip hop', '...   
28471  ['arkansas country', 'contemporary country', '...   
28472          ['adult standards', 'brill building pop']   
28473                                                NaN   
28474                                                NaN   

             spotify_track_id spotify_track_explicit  \
28470  1Mg13VMaQj4fhPtQT41vUZ                  False   
28471  3AiNYdRmhXcv7KCFHZOddA                  False   
28472  5w0dh39SYz1ei2tCs7X7Fo                  False   
28473                     NaN                    NaN   
28474                     NaN                    NaN   

       spotify_track_duration_ms  danceability  energy  loudness  speechiness  \
28470                   246000.0         0.817   0.625    -3.643       0.0719   
28471                   141946.0         0.637   0.963    -3.808       0.0770   
28472                   165560.0         0.685   0.760    -9.354       0.0430   
28473                        NaN           NaN     NaN       NaN          NaN   
28474                        NaN           NaN     NaN       NaN          NaN   

       acousticness  instrumentalness  valence    tempo  
28470        0.0508          0.001230    0.759  102.907  
28471        0.6620          0.000031    0.920  170.224  
28472        0.4300          0.000000    0.895  113.569  
28473           NaN               NaN      NaN      NaN  
28474           NaN               NaN      NaN      NaN

In [31]:
spotify_df['spotify_track_id'].value_counts()

0iA1unTbTbDOWUSlbwJ1pS    3
3mRM4NM8iO7UBqrSigCQFH    3
1zU2N6UlqvEHfwL6VjNLHf    2
6PYnUsNEpYC7A4BS2sjw3L    2
0DpjGs3EbsI1TKiYSSeTRF    2
                         ..
3cVdgpyCPWOmJOFc3Zsp6b    1
6bVB2MGR7LcotAIB1vfpw6    1
0yzAjMhKNNUWTPYn1zKRDl    1
695sRCJTtOHIY5ACQa7qtZ    1
3yWx3rzS2MDqkopz3utEOx    1
Name: spotify_track_id, Length: 23529, dtype: int64

## Initiate connexion with spotify API

In [32]:
cid = 'ed490024ea5f4072810146d4d3dff356'
secret = 'b74fa8fd014f4932bb15e79572a104fe'

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)

sp = client.Spotify(client_credentials_manager=client_credentials_manager)
sp.trace = False

### Create some utils functions

In [33]:
def getAudioFeatureSpotify(sp,tids):
    return sp.audio_features(tids)[0]

def keepGoodFeatures(features):
    newFeatures={
        'danceability': features['danceability'],
        'energy': features['energy'],
        'key': features['key'],
        'loudness': features['loudness'],
        'mode': features['mode'],
        'speechiness': features['speechiness'],
        'acousticness': features['acousticness'],
        'instrumentalness': features['instrumentalness'],
        'liveness': features['liveness'],
        'valence': features['valence'],
        'tempo': features['tempo'],
        'duration_ms': features['duration_ms'],
                }
    return newFeatures

def searchSpotify(titre):
    result = sp.search(titre,limit=1,type='track')['tracks']['items']
    if len(result) <=0:
        return result
    else :
        return result[0]

In [51]:
getTrackID=songArtistSongID.copy()
getTrackID['trackID']=0

for indexx,i in songArtistSongID.iterrows():
    song=i['Song']
    performer=i['Performer']
    
    q = song + ' ' + performer
    r=searchSpotify(q)
    
    if r != []:
        getTrackID.loc[indexx,'trackID']=r['id']

    
    time.sleep(0.05)
    
    if indexx % 500 == 0 and indexx > 10:
        print(round(indexx/len(songArtistSongID)*100,3),'%')
        
    if indexx % 1000 ==0 and indexx > 10:
        print(len(songArtistSongID)-len(getTrackID[getTrackID['trackID']==0]),"musiques trouve a l'index :",indexx)

1.756 %
3.512 %
684 musiques trouve a l'index : 1000
7.024 %
1240 musiques trouve a l'index : 2000
8.78 %
10.536 %
1780 musiques trouve a l'index : 3000
12.291 %
14.047 %
2446 musiques trouve a l'index : 4000
17.559 %
2793 musiques trouve a l'index : 5000
19.315 %
24.583 %
3502 musiques trouve a l'index : 7000
33.363 %
40.386 %
47.41 %
49.166 %
5879 musiques trouve a l'index : 14000
50.922 %
56.19 %
6772 musiques trouve a l'index : 16000
77.261 %
7212 musiques trouve a l'index : 22000
91.308 %
7646 musiques trouve a l'index : 26000
94.82 %
7910 musiques trouve a l'index : 27000
98.332 %
8354 musiques trouve a l'index : 28000
100.088 %
101.844 %
9196 musiques trouve a l'index : 29000
103.6 %
105.356 %
10068 musiques trouve a l'index : 30000
107.112 %
143.986 %
10816 musiques trouve a l'index : 41000
145.742 %
147.498 %
11662 musiques trouve a l'index : 42000
149.254 %
151.01 %
12480 musiques trouve a l'index : 43000
152.766 %
154.522 %
13295 musiques trouve a l'index : 44000
156.277 %
1

In [53]:
len(getTrackID[getTrackID['trackID']==0])

5491

In [54]:
getTrackID['trackID'].value_counts()

0                         5491
1hQFF33xi8ruavZNyovtUN       3
25leEEaz1gIpp7o21Fqyjo       3
2zEvB5gYtH1wICRBST2Qlg       2
4kV6opXYLOjYswb9G9liIp       2
                          ... 
2P1e3xpjDhYFqOFeu8IWzS       1
4ODWgs1CVAZKDbryEodjiz       1
4VZDv8sASBS8UruUBGTFdk       1
03j354P848KtNU2FVSwkDG       1
0cp4meeLcHubVHMnl1NJRm       1
Name: trackID, Length: 22842, dtype: int64

In [55]:
getTrackID

Song  \
0                                        Poor Little Fool   
1                                           One Sweet Day   
2       Candle In The Wind 1997/Something About The Wa...   
3                                     Do I Make You Proud   
4                                                       3   
...                                                   ...   
302081                          What'd You Come Here For?   
302093                              What's A Guy Gotta Do   
302108           What's A Matter Baby (Is It Hurting You)   
314894                 Everybody Dies In Their Nightmares   
320495                           Into The Great Wide Open   

                              Performer  \
0                          Ricky Nelson   
1            Mariah Carey & Boyz II Men   
2                            Elton John   
3                          Taylor Hicks   
4                        Britney Spears   
...                                 ...   
302081                   Trina & Tamara   
302093                      Joe Nichols   
302108                        Timi Yuro   
314894                     XXXTENTACION   
320495  Tom Petty And The Heartbreakers   

                                                   SongID  \
0                            Poor Little FoolRicky Nelson   
1                 One Sweet DayMariah Carey & Boyz II Men   
2       Candle In The Wind 1997/Something About The Wa...   
3                         Do I Make You ProudTaylor Hicks   
4                                         3Britney Spears   
...                                                   ...   
302081            What'd You Come Here For?Trina & Tamara   
302093                   What's A Guy Gotta DoJoe Nichols   
302108  What's A Matter Baby (Is It Hurting You)Timi Yuro   
314894     Everybody Dies In Their NightmaresXXXTENTACION   
320495  Into The Great Wide OpenTom Petty And The Hear...   

                       trackID  
0       5ayybTSXNwcarDtxQKqvWX  
1       0NEMfVkEjxSVjnG0sJms8v  
2       5LjSxAIKwyZvQqJ04ZQ0Da  
3       2jv9Ppwz9u3lqWfXV0q2sW  
4       6pymOcrCnMuCWdgGVTvUgP  
...                        ...  
302081  1Mg13VMaQj4fhPtQT41vUZ  
302093  3AiNYdRmhXcv7KCFHZOddA  
302108  5w0dh39SYz1ei2tCs7X7Fo  
314894  3GVkPk8mqxz0itaAriG1L7  
320495  17S4XrLvF5jlGvGCJHgF51  

[28475 rows x 4 columns]

In [ ]:
mySpotify_df = pd.DataFrame(columns=['SongID','SpotifyID','preview_url','release_date'])
songID_withName = pd.DataFrame(columns=['SongID','Performer','Song'])

songID=set()
for ii in billboard.iterrows():
    i=ii[1]
    if not i['SongID'] in songID:
        
        songID.add(i['SongID'])
        songID_withName.loc[i['SongID']]=pd.Series({'SongID':i['SongID'],
                                                 'Performer':i['Performer'],
                                                    'Song' : i['Song']})
        #q=searchSpotify(i['Song']+' '+i['Performer'])
        
        """
        if q != []:
            mySpotify_df.loc[i['SongID']]=pd.Series({'SongID':i['SongID'],
                                                 'SpotifyID':q['id'],
                                                 'preview_url':q['preview_url'],
                                                 'release_date':q['album']['release_date']})
        else :
            mySpotify_df.loc[i['SongID']]=pd.Series({'SongID':i['SongID'],
                                                 'SpotifyID':0,
                                                 'preview_url':0,
                                                 'release_date':0})"""
    #time.sleep(0.1)
    if ii[0] %1000 ==0:
        print(ii[0],i['SongID'])   
print(len(songID))

In [ ]:
for i in searched_spotify:
    for j in i:
        print(j)

print(xx['album']['release_date'])
print(xx['id'])
print(xx['preview_url'])

In [ ]:
xj['& in name']=xj['Performer'].apply(lambda x: '&' in x)
xj['is Costarring']=xj['Performer'].apply(lambda x: 'Co-Starring' in x)

xj['is With']=xj['Performer'].apply(lambda x: 'With' in x)

xj['is /']=xj['SongID'].apply(lambda x: '/' in x)

xj['is (']=xj['SongID'].apply(lambda x: '(' in x)

In [ ]:
def exploreDict(dic,level=-1):
    
    if type(dic) == type({}): 
        for i in dic.keys():
            level+=1
            
            print(level,'-'*level,i)
            
            if type(dic[i]) == type({}) or type(dic[i]) == type([]) :

                exploreDict(dic[i],level=level)
            else :
                level+=1
                print(level,'-'*level,dic[i])
                
    elif type(dic) == type([]):
        if len(dic) > 20 :
            print(level,'-'*level,dic)
        else :
            for i in range(len(dic)):
                if type(dic[i]) == type({}) or type(dic[i]) == type([]) :

                    exploreDict(dic[i],level=level)
                else :
                    print(level,'-'*level,dic[i])

In [ ]:
mySpotify_df = pd.DataFrame(columns=['SongID','SpotifyID','preview_url','release_date'])
songID_withName = pd.DataFrame(columns=['SongID','Performer','Song'])

songID=set()
for ii in billboard.iterrows():
    i=ii[1]
    if not i['SongID'] in songID:
        
        songID.add(i['SongID'])
        songID_withName.loc[i['SongID']]=pd.Series({'SongID':i['SongID'],
                                                 'Performer':i['Performer'],
                                                    'Song' : i['Song']})
        #q=searchSpotify(i['Song']+' '+i['Performer'])
        
        """
        if q != []:
            mySpotify_df.loc[i['SongID']]=pd.Series({'SongID':i['SongID'],
                                                 'SpotifyID':q['id'],
                                                 'preview_url':q['preview_url'],
                                                 'release_date':q['album']['release_date']})
        else :
            mySpotify_df.loc[i['SongID']]=pd.Series({'SongID':i['SongID'],
                                                 'SpotifyID':0,
                                                 'preview_url':0,
                                                 'release_date':0})"""
    #time.sleep(0.1)
    if ii[0] %1000 ==0:
        print(ii[0],i['SongID'])   
print(len(songID))

In [ ]:
songs_features_name_df=pd.merge(songID_withName,spotifyFeatures,how='left')

In [ ]:
songs_features_name_df.tail()

In [ ]:
mySpotify_df = pd.DataFrame(columns=['SongID','SpotifyID','preview_url','release_date'])

indexNAN=0
for ii in songs_features_name_df.iterrows():
    index=ii[0]
    i=ii[1]
    if not i['spotify_track_id'] == i['spotify_track_id']: # RETURN FALSE IF ITS NaN value
        indexNAN+=1
        if indexNAN %100 ==0 :
            print(indexNAN)
        
        q=searchSpotify(i['Song']+' '+i['Performer'])
        
        
        if q != []:
            mySpotify_df.loc[i['SongID']]=pd.Series({'SongID':i['SongID'],
                                                 'SpotifyID':q['id'],
                                                 'preview_url':q['preview_url'],
                                                 'release_date':q['album']['release_date']})
        else :
            mySpotify_df.loc[i['SongID']]=pd.Series({'SongID':i['SongID'],
                                                 'SpotifyID':0,
                                                 'preview_url':0,
                                                 'release_date':0})
    time.sleep(0.1)
    if ii[0] %1000 ==0:
        print(ii[0],i['SongID'])   
print(len(songID))

In [ ]:
xi=pd.merge(mySpotify_df,songID_withName,how='left')
xj=pd.merge(xi,artists_df,how='left')

In [ ]:
xj[xj['SpotifyID'] == 0]['is_featuring'].value_counts()
xj[xj['SpotifyID'] == 0][xj['is ('] == True]['is /'].value_counts()

len(xj['preview_url'].unique())
xj[xj['SpotifyID'] == 0].index

In [ ]:
xj[xj['SpotifyID'] == 0][xj['year_min'] > 2000]['year_min'].sort_values().value_counts()